In [ ]:
def dhcp_pyafq_pipeline(subject, session, local_env=False):
    """
    For each `subject`, `session` pair run tractography pipeline.
    systemctl --user start docker-desktop 
    0. Perpare local environment
    1. Generate isotropic DWI files
    2. Realign DWI to anatomy using rigid transformation
    3. Estimate CSD response function
    4. Reassign ribbon values
    5. Generate five-tissue-type (5TT) segmented tissue image
    6. Tractography
    7. Create BIDS derivatives
    8. Upload to s3
    
    Parameters
    ----------
    subject : string
    session : string
    aws_access_key : string
    aws_secret_key : string
    streamline_count : int
    local_env : boolean
    """
    
    import subprocess
    import os
    from os.path import exists, join
    import s3fs
    import json
    from AFQ.definitions.mapping import AffMap
      
    fs = s3fs.S3FileSystem()
    
    def print_quiet(string):
        """
        mrtrix doesn't provide a way to conviently supress progress messages without
        also supressing informational messages. 
        
        filter stderr/stdout before printing

        """
        import re
        
        # remove any line that looks like a progress message
        string = re.sub(r'.*\[[0-9 ]{3}\%\].*\n?', '', string)

        # remove empty lines
        string = re.sub(r'^\n$', '', string)
        
        # remove any leading or trailing whitespace
        string = string.strip()
        
        if string != "":
            print(string, flush=True)
        
    
    ###############################################################################
    # Step 0: Prepare local environment
    ###############################################################################
    
    print('Step 0: Prepare local environment', flush=True)
    
    os.makedirs(join('input', f'sub-{subject}', f'ses-{session}'), exist_ok=True)
    os.makedirs(join('output', f'sub-{subject}', f'ses-{session}'), exist_ok=True)
    
    
    ###############################################################################
    # Step 1: Loading pediatric templates
    ###############################################################################
    
    print('Step 1: Loading pediatric templates', flush=True)

    import os
    os.makedirs(join('output', f'sub-{subject}', f'ses-{session}'), exist_ok=True)

    from AFQ.api.participant import ParticipantAFQ
    import AFQ.api.bundle_dict as abd
    import AFQ.data.fetch as afd
    import os.path as op

    nifti = join('input', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.nii.gz')
    bval = join('input', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.bval')
    bvec = join('input', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-preproc_resliced_aligned_dwi.bvec')

    fs.get((f'dhcp-afq/mrtrix/sub-{subject}/ses-{session}/' f'sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.nii.gz'), join('input', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.nii.gz'))
    brainmask = join('input', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-preproc_space_dwi_resliced_aligned_brainmask.nii.gz')
    
    fs.get((f'dhcp-afq/mrtrix/sub-{subject}/ses-{session}/' f'sub-{subject}_ses-{session}_desc-csd_tractography.tck'), join('input', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-csd_tractography.tck'))
    tractogram = join('input', f'sub-{subject}', f'ses-{session}', f'sub-{subject}_ses-{session}_desc-csd_tractography.tck')
    
    myafq = ParticipantAFQ(
        # ==== BIDS parameters ====
        nifti,
        bval,
        bvec,
        f'output/sub-{subject}/ses-{session}',
        # ===== Registration parameters ====
        reg_template_spec=afd.read_pediatric_templates()["UNCNeo-withCerebellum-for-babyAFQ"],
        reg_subject_spec="b0",
        scalars=["dti_fa", "dti_md", "dki_fa", "dki_md", "dki_mk"],
        brain_mask=brainmask,
        mapping_definition=AffMap(),
        # ==== Bundle parameters ====
        bundle_info=abd.baby_bd(),
        # ==== Tracking parameters ====
        import_tract=tractogram,
        # ==== Segmentation parameters ====
        segmentation_params={
            "filter_by_endpoints": False, "parallel_segmentation": {"engine": "serial"}, 'dist_to_waypoint': 0.5},
        clean_params={
            'distance_threshold': 3},
    )
    
    ###############################################################################
    # Step 2: Run pyAFQ
    ###############################################################################
    
    print('Step 2: Run pyAFQ', flush=True)

    myafq.export("template_xform")

    myafq.clobber(dependent_on="recog")

    myafq.export_all()